# Diamond Price Prediction: Machine Learning & Statistical Analysis

This notebook demonstrates:
- Loss functions for model evaluation
- Bayesian statistics and uncertainty quantification
- Machine learning pipeline with train/test splits
- Predictive analytics with multiple algorithms

In [ ]:
# Enhanced imports for machine learning and Bayesian analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Load and explore the diamond dataset
directory = "C:/Users/vgwis/Documents/Code/Python/Personal/Analytics/"
df = pd.read_csv(directory + 'Diamonds.csv')

# Clean column names
for column in df.columns:
    column_s = column.strip()
    df = df.rename(columns={column: column_s})

print("=== DIAMOND DATASET ANALYSIS ===")
print(f"Dataset shape: {df.shape}")
print(f"\nDataset info:")
print(df.info())
print(f"\nFirst few rows:")
print(df.head())
print(f"\nMissing values per column:")
print(df.isnull().sum())
print(f"\nBasic statistics:")
print(df.describe())

In [ ]:
# Data Preprocessing for Machine Learning
df_clean = df.copy()

# Remove the unnamed index column if it exists
if 'Unnamed: 0' in df_clean.columns:
    df_clean = df_clean.drop('Unnamed: 0', axis=1)

# Remove outliers (diamonds with zero dimensions)
df_clean = df_clean[(df_clean.x > 0) & (df_clean.y > 0) & (df_clean.z > 0)]

# Encode categorical variables
label_encoders = {}
categorical_columns = ['cut', 'color', 'clarity']

for col in categorical_columns:
    le = LabelEncoder()
    df_clean[col + '_encoded'] = le.fit_transform(df_clean[col])
    label_encoders[col] = le

print("Categorical encoding completed:")
for col in categorical_columns:
    print(f"{col}: {dict(zip(label_encoders[col].classes_, range(len(label_encoders[col].classes_))))}")

print(f"\nCleaned dataset shape: {df_clean.shape}")

In [ ]:
# Train-Test Split
# Features: carat, depth, table, x, y, z, and encoded categorical variables
feature_columns = ['carat', 'depth', 'table', 'x', 'y', 'z', 
                   'cut_encoded', 'color_encoded', 'clarity_encoded']
target_column = 'price'

X = df_clean[feature_columns]
y = df_clean[target_column]

# Split into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=pd.qcut(y, 5, duplicates='drop')
)

# Scale features for better model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("=== TRAIN-TEST SPLIT COMPLETED ===")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")
print(f"Features used: {feature_columns}")
print(f"Target variable: {target_column}")
print(f"\nTarget variable statistics:")
print(f"Training set - Mean: ${y_train.mean():.2f}, Std: ${y_train.std():.2f}")
print(f"Testing set - Mean: ${y_test.mean():.2f}, Std: ${y_test.std():.2f}")

In [ ]:
# Model Implementation and Training
print("=== TRAINING MULTIPLE MODELS ===")

# 1. Linear Regression
print("Training Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
lr_predictions = lr_model.predict(X_test_scaled)

# 2. Random Forest Regressor
print("Training Random Forest...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)  # Random Forest doesn't need scaling
rf_predictions = rf_model.predict(X_test)

# 3. Bayesian Ridge Regression (for uncertainty quantification)
print("Training Bayesian Ridge Regression...")
br_model = BayesianRidge()
br_model.fit(X_train_scaled, y_train)
br_predictions, br_std = br_model.predict(X_test_scaled, return_std=True)

print("All models trained successfully!")

In [ ]:
# Loss Functions and Model Evaluation
def calculate_loss_functions(y_true, y_pred, model_name):
    """Calculate various loss functions for model evaluation"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # Huber Loss (less sensitive to outliers)
    huber_loss = np.mean(np.where(np.abs(y_true - y_pred) <= 1.35, 
                                 0.5 * (y_true - y_pred)**2,
                                 1.35 * (np.abs(y_true - y_pred) - 0.675)))
    
    return {
        'Model': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE (%)': mape,
        'Huber Loss': huber_loss
    }

print("=== LOSS FUNCTION ANALYSIS ===")

# Calculate metrics for all models
models_performance = []
models_performance.append(calculate_loss_functions(y_test, lr_predictions, 'Linear Regression'))
models_performance.append(calculate_loss_functions(y_test, rf_predictions, 'Random Forest'))
models_performance.append(calculate_loss_functions(y_test, br_predictions, 'Bayesian Ridge'))

# Create performance comparison DataFrame
performance_df = pd.DataFrame(models_performance)
print(performance_df.round(2))

# Find best model for each metric
print("\n=== BEST MODEL BY METRIC ===")
for metric in ['MSE', 'RMSE', 'MAE', 'R²', 'MAPE (%)', 'Huber Loss']:
    if metric == 'R²':
        best_idx = performance_df[metric].idxmax()
    else:
        best_idx = performance_df[metric].idxmin()
    best_model = performance_df.loc[best_idx, 'Model']
    best_value = performance_df.loc[best_idx, metric]
    print(f"{metric}: {best_model} ({best_value:.2f})")

In [ ]:
# Comprehensive Visualizations
plt.figure(figsize=(20, 15))

# 1. Actual vs Predicted scatter plots
plt.subplot(3, 3, 1)
plt.scatter(y_test, lr_predictions, alpha=0.5, s=10)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Price ($)')
plt.ylabel('Predicted Price ($)')
plt.title('Linear Regression: Actual vs Predicted')
plt.text(0.05, 0.95, f'R² = {r2_score(y_test, lr_predictions):.3f}', 
         transform=plt.gca().transAxes, verticalalignment='top')

plt.subplot(3, 3, 2)
plt.scatter(y_test, rf_predictions, alpha=0.5, s=10, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Price ($)')
plt.ylabel('Predicted Price ($)')
plt.title('Random Forest: Actual vs Predicted')
plt.text(0.05, 0.95, f'R² = {r2_score(y_test, rf_predictions):.3f}', 
         transform=plt.gca().transAxes, verticalalignment='top')

plt.subplot(3, 3, 3)
plt.scatter(y_test, br_predictions, alpha=0.5, s=10, color='purple')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Price ($)')
plt.ylabel('Predicted Price ($)')
plt.title('Bayesian Ridge: Actual vs Predicted')
plt.text(0.05, 0.95, f'R² = {r2_score(y_test, br_predictions):.3f}', 
         transform=plt.gca().transAxes, verticalalignment='top')

# 2. Residual plots
plt.subplot(3, 3, 4)
residuals_lr = y_test - lr_predictions
plt.scatter(lr_predictions, residuals_lr, alpha=0.5, s=10)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Price ($)')
plt.ylabel('Residuals ($)')
plt.title('Linear Regression: Residual Plot')

plt.subplot(3, 3, 5)
residuals_rf = y_test - rf_predictions
plt.scatter(rf_predictions, residuals_rf, alpha=0.5, s=10, color='green')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Price ($)')
plt.ylabel('Residuals ($)')
plt.title('Random Forest: Residual Plot')

plt.subplot(3, 3, 6)
residuals_br = y_test - br_predictions
plt.scatter(br_predictions, residuals_br, alpha=0.5, s=10, color='purple')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Price ($)')
plt.ylabel('Residuals ($)')
plt.title('Bayesian Ridge: Residual Plot')

# 3. Loss function comparison
plt.subplot(3, 3, 7)
metrics = ['MSE', 'RMSE', 'MAE', 'MAPE (%)']
x_pos = np.arange(len(metrics))
width = 0.25

for i, model in enumerate(['Linear Regression', 'Random Forest', 'Bayesian Ridge']):
    values = [performance_df.loc[i, metric] for metric in metrics]
    # Normalize values for better comparison
    normalized_values = [val / performance_df[metric].max() for val, metric in zip(values, metrics)]
    plt.bar(x_pos + i*width, normalized_values, width, 
            label=model, alpha=0.8)

plt.xlabel('Metrics')
plt.ylabel('Normalized Score (lower is better)')
plt.title('Model Performance Comparison')
plt.xticks(x_pos + width, metrics)
plt.legend()

# 4. Feature importance (Random Forest)
plt.subplot(3, 3, 8)
feature_importance = rf_model.feature_importances_
sorted_indices = np.argsort(feature_importance)[::-1]
plt.bar(range(len(feature_importance)), feature_importance[sorted_indices])
plt.xticks(range(len(feature_importance)), 
           [feature_columns[i] for i in sorted_indices], rotation=45)
plt.ylabel('Importance')
plt.title('Random Forest: Feature Importance')

# 5. Prediction distribution
plt.subplot(3, 3, 9)
plt.hist(y_test, bins=50, alpha=0.5, label='Actual', density=True)
plt.hist(rf_predictions, bins=50, alpha=0.5, label='RF Predicted', density=True)
plt.hist(br_predictions, bins=50, alpha=0.5, label='BR Predicted', density=True)
plt.xlabel('Price ($)')
plt.ylabel('Density')
plt.title('Price Distribution: Actual vs Predicted')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Bayesian Analysis and Uncertainty Quantification
print("=== BAYESIAN STATISTICS CONCEPTS ===")

# 1. Uncertainty estimation with Bayesian Ridge
print("1. UNCERTAINTY ESTIMATION")
print(f"Mean prediction uncertainty (std): ${br_std.mean():.2f}")
print(f"Max prediction uncertainty: ${br_std.max():.2f}")
print(f"Min prediction uncertainty: ${br_std.min():.2f}")

# 2. Confidence intervals for predictions
confidence_level = 0.95
z_score = stats.norm.ppf((1 + confidence_level) / 2)
lower_bound = br_predictions - z_score * br_std
upper_bound = br_predictions + z_score * br_std

# Calculate coverage (how many actual values fall within confidence intervals)
coverage = np.mean((y_test >= lower_bound) & (y_test <= upper_bound))
print(f"\n2. CONFIDENCE INTERVALS")
print(f"{confidence_level*100}% Confidence Interval Coverage: {coverage:.3f}")
print(f"Expected coverage: {confidence_level:.3f}")

# 3. Bayesian vs Frequentist interpretation
print(f"\n3. BAYESIAN INTERPRETATION")
print("Bayesian Ridge provides:")
print("- Posterior distribution over model parameters")
print("- Natural uncertainty quantification")
print("- Regularization through prior distributions")
print(f"- Alpha (precision of noise): {br_model.alpha_:.2e}")
print(f"- Lambda (precision of weights): {br_model.lambda_:.2e}")

In [ ]:
# Visualize Bayesian uncertainty
plt.figure(figsize=(15, 10))

# Sample some predictions to visualize
sample_size = 1000
sample_indices = np.random.choice(len(y_test), sample_size, replace=False)
y_test_sample = y_test.iloc[sample_indices]
br_pred_sample = br_predictions[sample_indices]
br_std_sample = br_std[sample_indices]
lower_sample = lower_bound[sample_indices]
upper_sample = upper_bound[sample_indices]

# Sort by actual values for better visualization
sort_indices = np.argsort(y_test_sample)
y_test_sorted = y_test_sample.iloc[sort_indices]
br_pred_sorted = br_pred_sample[sort_indices]
lower_sorted = lower_sample[sort_indices]
upper_sorted = upper_sample[sort_indices]

plt.subplot(2, 2, 1)
x_range = range(len(y_test_sorted))
plt.fill_between(x_range, lower_sorted, upper_sorted, alpha=0.3, label='95% Confidence Interval')
plt.plot(x_range, y_test_sorted, 'o', markersize=2, label='Actual', alpha=0.7)
plt.plot(x_range, br_pred_sorted, 'r-', linewidth=1, label='Predicted')
plt.xlabel('Sample Index (sorted by actual price)')
plt.ylabel('Price ($)')
plt.title('Bayesian Predictions with Uncertainty')
plt.legend()

# Uncertainty vs prediction accuracy
plt.subplot(2, 2, 2)
absolute_errors = np.abs(y_test - br_predictions)
plt.scatter(br_std, absolute_errors, alpha=0.5, s=10)
plt.xlabel('Prediction Uncertainty (std)')
plt.ylabel('Absolute Error ($)')
plt.title('Uncertainty vs Prediction Error')
correlation = np.corrcoef(br_std, absolute_errors)[0, 1]
plt.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
         transform=plt.gca().transAxes, verticalalignment='top')

# Histogram of uncertainties
plt.subplot(2, 2, 3)
plt.hist(br_std, bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Prediction Uncertainty (std)')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Uncertainties')

# High vs low uncertainty predictions
plt.subplot(2, 2, 4)
high_uncertainty_mask = br_std > np.percentile(br_std, 75)
low_uncertainty_mask = br_std < np.percentile(br_std, 25)

plt.scatter(y_test[low_uncertainty_mask], br_predictions[low_uncertainty_mask], 
           alpha=0.5, s=10, label='Low Uncertainty', color='blue')
plt.scatter(y_test[high_uncertainty_mask], br_predictions[high_uncertainty_mask], 
           alpha=0.5, s=10, label='High Uncertainty', color='red')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Price ($)')
plt.ylabel('Predicted Price ($)')
plt.title('Predictions by Uncertainty Level')
plt.legend()

plt.tight_layout()
plt.show()

print(f"\n4. INSIGHTS FROM UNCERTAINTY ANALYSIS")
high_uncertainty_error = np.mean(absolute_errors[high_uncertainty_mask])
low_uncertainty_error = np.mean(absolute_errors[low_uncertainty_mask])
print(f"Average error for high uncertainty predictions: ${high_uncertainty_error:.2f}")
print(f"Average error for low uncertainty predictions: ${low_uncertainty_error:.2f}")
print(f"Error reduction ratio: {high_uncertainty_error/low_uncertainty_error:.2f}x")

# Summary: Statistical and Computing Concepts Demonstrated

## Statistical Concepts Covered:

### 1. **Loss Functions**
- **MSE (Mean Squared Error)**: Penalizes large errors quadratically
- **RMSE (Root Mean Squared Error)**: Same units as target variable
- **MAE (Mean Absolute Error)**: Less sensitive to outliers
- **MAPE (Mean Absolute Percentage Error)**: Scale-independent metric
- **Huber Loss**: Robust to outliers (combines MSE and MAE)
- **R² Score**: Explained variance ratio

### 2. **Bayesian Statistics**
- **Posterior Distributions**: BayesianRidge provides uncertainty estimates
- **Prior Knowledge**: Regularization through precision parameters
- **Uncertainty Quantification**: Standard deviations for predictions
- **Confidence Intervals**: 95% coverage analysis
- **Bayesian vs Frequentist**: Probabilistic vs point estimates

## Computing/ML Concepts Covered:

### 1. **Machine Learning Pipeline**
- Data preprocessing and cleaning
- Feature encoding (categorical variables)
- Train-test split with stratification
- Feature scaling for algorithms that require it

### 2. **Predictive Analytics**
- **Linear Regression**: Baseline linear model
- **Random Forest**: Ensemble method with feature importance
- **Bayesian Ridge**: Probabilistic linear model with uncertainty

### 3. **Model Evaluation**
- Multiple metrics comparison
- Residual analysis
- Feature importance analysis
- Prediction distribution comparison
- Uncertainty-error correlation analysis

## Key Insights:
- Random Forest typically performs best on this dataset
- Bayesian methods provide valuable uncertainty estimates
- Higher uncertainty correlates with higher prediction errors
- Feature importance shows carat weight dominates pricing